# Проект для «Викишоп» с BERT.

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

## Загрузка данных.

Загрузим необходимые библиотеки.

In [ ]:
#скачивание библиотеки
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#загрузка библиотек
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import torch
import transformers as ppb
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Сохраним датасет в переменной.

In [ ]:
#загрузка датасета
try:
    df = pd.read_csv("/datasets/toxic_comments.csv", on_bad_lines='skip')
except:
    df = pd.read_csv("toxic_comments.csv", on_bad_lines='skip')

Выведем датасет на экран.

In [ ]:
#вывод таблицы
df.head()


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Получим краткую информацию о датасете.

In [ ]:
#вывод информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [ ]:
#вывод размера таблицы
df.shape

(159571, 2)

##Краткие выводы:

Таблица содержит 2 столбца с признаками. Обучающий признак содержит строки с текстом которые в дальнейшем переведём в векторный вид по средством BERT. Целевой признак содержит категориальные значения, поэтому применим F-меру для достижения полноты и точности.

## Подготовка

Инициируем модель, токенайзер и словари.

In [ ]:
#инициализациия модели,токенизатора, базы данных для работы с BERT.
model_class, tokenizer_class, model_name = (ppb.AutoModel, ppb.AutoTokenizer, 'unitary/toxic-bert')

Предобучим модель и токенизатор.

In [ ]:
#предобучение модели и токенизатора из базы
tokenizer = tokenizer_class.from_pretrained(model_name)
model = model_class.from_pretrained(model_name)


Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Для построения модели нам понадобится не все данные из таблицы. Поэтому урежем датасет до 20000.  
Так же создадим тестовую выборку.

In [ ]:
#сокращение датасета и разделение на признаки.
train_df  = df[:20000]
features_train = train_df["text"]
target_train = train_df["toxic"]

test_df = df[20000:23000]
features_test = test_df["text"]
target_test = test_df["toxic"]

Проверким баланс классов.

In [ ]:
#подсчет количества значений каждого класса
target_train.value_counts()

0    17940
1     2060
Name: toxic, dtype: int64

Как видно что у нас присутствует дисбаланс классов.  
Для нормального обучения модели применим метод даунсемплинга.

Напишем функцию.

In [ ]:
#функция уменьшения количества большего класса
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled


Применим функцию.

In [ ]:
#создание новых переменных
features_downsampled, target_downsampled = downsample(features_train, target_train, 0.12)

Проверка изменений.

In [ ]:
#вывод количества значений классов
target_downsampled.value_counts()

0    2153
1    2060
Name: toxic, dtype: int64

При разделении на батчи у нас образуются лишние данные, чтобы этого избежать округрлим до сотых.

In [ ]:
features_downsampled = features_downsampled[:4200]
target_downsampled = target_downsampled[:4200]

Приведем текст в токены. При этом ограничим длинну для того чтобы расходовалось меньше памяти. Добавим токены начала и конца.

In [ ]:
#перевод данных в токены
tokenized = features_downsampled.apply(lambda x: tokenizer.encode(x, 
                                                                   add_special_tokens=True, 
                                                                   max_length = 150,
                                                                   truncation = True))

In [ ]:
#вывод токенизированных данных
tokenized.head()

5873     [101, 1000, 2092, 1045, 2453, 5136, 1998, 2059...
3846     [101, 16780, 1010, 2417, 1998, 2304, 1012, 129...
3306     [101, 1000, 2748, 1045, 2018, 2023, 2168, 3291...
17437    [101, 2928, 2004, 6528, 2003, 1037, 22418, 224...
8609     [101, 1045, 4033, 1005, 1056, 2018, 2051, 2000...
Name: text, dtype: object

In [ ]:
tokenized.shape

(4200,)

Применим метод padding для уравнения длин текстов.  
Заполним отступы нулями.

In [ ]:

#применение метода
padded = np.array([i + [0]*(150 - len(i)) for i in tokenized.values])

Чтобы показать модели что нулевые значения не несут значимой информации создадим маску.

In [ ]:
#создание маски
attention_mask = np.where(padded != 0, 1, 0)


Т.к данных много и преобразование займёт много оперативной памяти, то разделим выборку на батчи и обработаем их по частям.
Преобразуем данные в многомерные вектора("тензоры").  
Далее указываем что гдадиенты не нужны.

In [ ]:
#обработка данных путём разделения на батчи
batch_size = 100
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.tensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.tensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/42 [00:00<?, ?it/s]

Объединяем данные и создаем признаки.

In [ ]:
#создание признаков
features_train = np.concatenate(embeddings)

## Обучение

Применим модель логистической регрессии для построения предсказаний.  Применим кросс-валидацию на тренировочной выборке.

In [ ]:
#обучение модели и применение кросс валидации.
log_model = LogisticRegression(max_iter =1000, random_state = 12345)
log_model.fit(features_train,target_downsampled)
scores = cross_val_score(log_model, features_train, target_downsampled, scoring = "f1", cv = 3)

print("F-мера:",(scores).mean())

F-мера: 0.9710751635431195


Преобразуем данные тестовой выборки тем же способом.

In [ ]:
tokenized_test = features_test.apply(lambda x: tokenizer.encode(x,     
                                                           add_special_tokens=True, 
                                                           max_length = 150,
                                                           truncation = True))

padded_test = np.array([i + [0]*(150 - len(i)) for i in tokenized_test.values])

attention_mask_test = np.where(padded_test != 0, 1, 0)

embeddings_test = []

for i in notebook.tqdm(range(padded_test.shape[0] // batch_size)):
    batch_test = torch.tensor(padded_test[batch_size*i:batch_size*(i+1)])
    attention_mask_batch_test = torch.tensor(attention_mask_test[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings_test = model(batch_test, attention_mask=attention_mask_batch_test)

    embeddings_test.append(batch_embeddings_test[0][:,0,:].numpy())

features_test = np.concatenate(embeddings_test)

  0%|          | 0/30 [00:00<?, ?it/s]

Применим модель на тестовой выборке и выведем результат.

In [ ]:
predictions = log_model.predict(features_test)

print("F-мера:",f1_score(target_test,predictions))

F-мера: 0.9074074074074073
